In [2]:
import nibabel as nib
import pandas as pd
import numpy as np
import os

In [3]:
def cifti2_filename(cifti1_path, output_dir):
    basename = os.path.basename(cifti1_path)

    # List of known CIFTI suffixes
    known_suffixes = [".dtseries.nii", ".dconn.nii", ".dscalar.nii", ".dpconn.nii"]

    # Match and strip full suffix
    for suffix in known_suffixes:
        if basename.endswith(suffix):
            stem = basename[: -len(suffix)]  # strip the full suffix
            cifti2_name = f"{stem}_cifti2{suffix}"
            return os.path.join(output_dir, cifti2_name)

    raise ValueError("Unknown CIFTI suffix in filename")

In [ ]:
def calc_correlations(input_file, tmask):
    if input_file.endswith(".dtseries.nii"):
        cifti_img = nib.load(input_file)


In [9]:
example_cifti_cohort_filepath = "./data/cohort_files/cohortfiles_washu120.txt"
example_tmask_cohort_filepath = "./data/tmask_files/tmasklist_washu120.txt"

cifti_cohort_df = pd.read_csv(example_cifti_cohort_filepath, delim_whitespace=True, header=None)
tmask_cohort_df = pd.read_csv(example_tmask_cohort_filepath, delim_whitespace=True, header=None)

example_idx = 111
subj, cifti1_path = cifti_cohort_df.iloc[example_idx, :2].tolist()
tmask_subj, tmask_path = tmask_cohort_df.iloc[example_idx, :].tolist()
assert subj == tmask_subj
tmask = np.loadtxt(tmask_path, dtype=int)

cifti2_path = cifti2_filename(cifti1_path, "./data/washu120/")

cifti_img = nib.load(cifti2_path)
cifti_data = cifti_img.get_fdata()
cifti_header = cifti_img.header
cifti_axes = cifti_header.get_axis(0)
print(cifti_axes)